#### Idei:
* x Overfitting: schimbat dropout
* x Mai multe batch-uri pentru training online
* x Verificat ca ia corect screenshots pentru state si next state
* x Scoate actiunea "0"
* x 'board' din functia makeMove e acum var globala, nu il mai cauta de fiecare data, dar sarpele tot mai sare pozitii uneori. Pot face schimbarea asta si la 'state' si 'score'
* x Verificat daca fullTarget updateaza cum trebuie doar actiunea potrivita - Pare ok
* x Verificat daca e un offset la stari atunci cand faci mutari conform modelului - Da, e offset
* x Puncte constante pentru reward (100p)
* x MakeMove doar din 2 in 2
* x Downsampling
* x De verificat in continuare daca se updateaza cum trebuie weights - Pare ca da, am verificat cu 3 epochs x 300 exp. Weights se schimba, dar doar putin si doar unele din centru
* x Verificat de ce modelul pare a avea cam aceleasi rezultate in orice pozitie - Clar o problema majora era ca simplificasem prea mult modelul (nu e ok sa fie MSE f mic din prima); O alta problema majora a fost Learning Rate prea mare (l-am scazut la 0.0001)
* x Adaugat strides la conv2d, cu stride = kernel / 2
* x Renunta la pooling layers (?)
* x Cea mai mare problema nu e ca tot incearca aceeasi actiune, ci ca nu invata sa evite peretele la ultima actiune - De verificat, dar posibil problema era ca nu punea la Experience -1000 la ultima cum trebuie
* x Input cu diferenta dintre doua screenshots pentru directie, in loc de ambele screenshots - Nu merge pentru Snake, pentru ca cercurile recompena raman pe loc si ar disparea la diferenta. Merge doar pentru jocuri unde tot ce conteaza se misca.
* x Fixed Q target pentru offline training


* Fixed Q target pentru online training
* Double DNN
* Gamma mai mare


* Considerat invatare online doar pe ultimele exemple (max memory = 32)

#### Dezvoltare:
* PrevAction ca input pentru DNN 
    - Queues: 4 screens, 3 moves, 2 rewards (if moves > 1) 
    - La fiecare adaugare de un nou reward se poate memora un nou Experience
    - Imaginile salvate la final din Episode
    - saveImg trebuie sa contina si prevAction
    - pentru joc efectiv ai nevoie doar de primele 2 screens din queue

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
import time
import os
import random
import io
from PIL import Image, ImageOps
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import glob
from collections import deque

set_image_data_format('channels_first')

Using TensorFlow backend.


#### Parameters

In [44]:
STILL_ALIVE_REWARD = 0
DEAD_REWARD = -1
SCORE_REWARD = 1

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 100
RESIZE_HEIGHT = 71
READ_BATCH = 2

MOVES = 4
RIGHT = 0
DOWN = 1
LEFT = 2
UP = 3

SHOTS_FOLDER = 'data/shots/'
EXP_FOLDER = 'data/experience/'

SAMPLE_ALIVE = 1
SAMPLE_DEAD = 5
SAMPLE_REWARD = 20

MAX_MEMORY = 500

GAMMA = 0.9
LEARNING_RATE = 0.001
EPS_INIT = 1.0
EPS_MIN = 0.05
EPS_DECAY = 0.998
CLONE_STEPS = 6500

#### Browser functions

In [3]:
def selectLevel(l):
    xpath = '/html/body/section/div[2]/nav/p[' + str(l+1) + ']'
    
    level = browser.find_element_by_xpath(xpath)
    level.click()
    
    
def clickBoard(board):        
    try:
        board.click()
    except WebDriverException:
        print('Exception')
        return
    
    
def saveScreen(fileName):    
    state = getState()
#     if state == 'playing' or state == 'paused':    
    ss = browser.get_screenshot_as_file(SHOTS_FOLDER + fileName)
    
    
def getScreen():
    ss = browser.get_screenshot_as_png()
    im = Image.open(io.BytesIO(ss))
    
    im = preprocImg(im)
    return np.asarray(im.convert("L"))
    
    
def getState():
    xpath = '/html/body/section/div[2]'
    state = browser.find_element_by_xpath(xpath)
    c = state.get_attribute('class')
    
    return c.split(' ')[-1]


def getScore():
    state = getState()
    
    if state == 'playing' or state == 'paused':
        
        xpath = '/html/body/section/div[2]/p[1]/span'
        score = browser.find_element_by_xpath(xpath)

        if not score.text.isnumeric():
            return 0
        return int(score.text)
    
    return 0


def getBoard():
    xpath = '/html/body/section/div[2]/div'
    board = browser.find_element_by_xpath(xpath)
    
    return board


def makeMove(board, m):    
    
    if m == RIGHT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_RIGHT)
    elif m == DOWN:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_DOWN)
    elif m == LEFT:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_LEFT)
    elif m == UP:
        browser.find_element_by_tag_name('body').send_keys(Keys.ARROW_UP)
        

#### Preprocess functions

In [4]:

def cropImgOld(fileName):
    ss = plt.imread(fileName)

    ss = ss[120:659, 273:1023, :]
    
    plt.imsave(fileName, ss)


def saveImage(arr, file):
    '''Save an image array to a file'''
    img = Image.fromarray(arr.astype(np.uint8))
    img.save(file)
        

def preprocImg(im):
    '''Preprocess a screenshot.'''     
    # Crop board
    im = im.crop((273, 120, 1023, 659))
    
    # Grayscale
    im = ImageOps.grayscale(im)
    
    # Binarization
    t = 127
    im = im.point(lambda x: 255 if x > t else 0)
    
    # Resize
    im = im.resize((RESIZE_WIDTH, RESIZE_HEIGHT))
    
    return im
    
    
def preprocAll(gameIndex):
    '''Preprocess all screenshots in the data folder.'''
    files = os.listdir(SHOTS_FOLDER)
    for f in files:
        g = parseName(f)[0]
        
        if g >= gameIndex:        
            fileName =  SHOTS_FOLDER + f
            im = Image.open(fileName)   
            im = preprocImg(im)
            im.save(fileName)
            

### Q model

#### DNN action output

Functional API, citeste batch-uri de imagini, considera actiunea ca output. 

In [5]:

def nn_model_actionoutput():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
#     conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(32, (4, 4), activation='relu', name='conv3')(pool)
#     conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.0, name='dropout1')(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.0, name='dropout2')(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    optimizer = Adam(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


def nn_model_nopool():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.
    Does not use pooling layers, only strides.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), strides=(2, 2), activation='relu', name='conv1')(screen_input)    
    conv = Conv2D(32, (4, 4), strides=(2, 2), activation='relu', name='conv2')(conv)    
    conv = Conv2D(16, (4, 4), strides=(2, 2), activation='relu', name='conv3')(conv)
    
    flat = Flatten(name='flat')(conv)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(rate=0.0, name='dropout1')(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(rate=0.0, name='dropout2')(dense)
    
    output = Dense(MOVES, activation='linear', name='output')(drop)
    
    model = Model(inputs=screen_input, outputs=output)
    
    optimizer = Adam(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


def nn_model_deepmind():
    '''Full NN model, using the Keras Functional API.
    Action as output, final layer has as many neurons as possible actions.
    The structure of the NN is the same as the initial one used by DeepMind.'''
    
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(32, (8, 8), strides=(4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(64, (4, 4), strides=(2, 2), activation='relu', name='conv2')(conv)
    conv = Conv2D(64, (3, 3), activation='relu', name='conv3')(conv)
    
    flat = Flatten(name='flat')(conv)
    dense = Dense(512, activation='relu', name='dense')(flat)    
    output = Dense(MOVES, activation='linear', name='output')(dense)
    
    model = Model(inputs=screen_input, outputs=output)
    
    optimizer = RMSprop(lr=LEARNING_RATE)
    
    model.compile(loss='mean_squared_error',
                 optimizer = optimizer)
    
    return model


#### Experience class and functions

In [6]:

class Experience:
    '''Class for an experience.'''    
    
    def __init__(self, state=[], action=0, label=0, nextState=[]):
        self.state = state
        self.action = action
        self.label = label
        self.nextState = nextState
        
        
    def save(self, folder, index):
        '''Save experience to a file.'''

        file = folder + str(index).zfill(4) + '_' + str(self.action) + '_' + str(self.label) + '.xp'
        f = open(file, 'w')

        for s in self.state:
            list = s.flatten()
            for l in list:
                f.write(str(l) + ' ')
            f.write('\n')

        for s in self.nextState:
            list = s.flatten()
            for l in list:
                f.write(str(l) + ' ')
            f.write('\n')

        f.close()


    def readArray(self, line):
        '''Read an array representing an screenshot from a file.'''
        elems = line.split(' ')[:-1]

        a = []
        for e in elems:
            a.append(int(e))

        a = np.asarray(a)
        a = a.reshape(RESIZE_HEIGHT, RESIZE_WIDTH)

        return a

    
    def read(self, folder, file):
        '''Read an experience from a file.'''
        f = open(folder + file, 'r')

        elems = parseExpName(file)
        self.action = int(elems[1])
        self.label = int(elems[2])

        s1 = self.readArray(f.readline())
        s2 = self.readArray(f.readline())
        self.state = [s1, s2]

        ns1 = self.readArray(f.readline())
        ns2 = self.readArray(f.readline())
        self.nextState = [ns1, ns2]

        f.close()
                        
            
def rememberExp(memory, exps):
    '''Add new experiences to the memory.'''
    for e in exps:
        if len(memory) == MAX_MEMORY:
            memory.pop(0)
        memory.append(e)


def sampleMemory(memory, nSamples, propD, propR, propA):
    '''Sample experiences from the memroy according to some proportions.
    
    Parameters
    ----------
    memory: the remembered experiences
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    nSamples = min(len(memory), nSamples)
        
    perc = []
    nd, na, nr = (0, 0, 0)
    for exp in memory:
        r = exp.label
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(memory, size=nSamples, replace=False, p=perc)
    return samples


def parseExpName(file):
    return file[:-3].split('_')


def sampleFiles(nSamples,
                folder=EXP_FOLDER,
                propD=SAMPLE_DEAD, 
                propR=SAMPLE_REWARD,
                propA=SAMPLE_ALIVE):
    '''Sample the experience files according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    folder: the experience folder
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(folder)
    nSamples = min(nSamples, len(files))
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseExpName(f)[2]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamples(samples):
    '''Read samples from experience files.'''
    states = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):    
        exp = Experience()
        exp.read(EXP_FOLDER, file)
            
        states.append(exp.state)
        actions.append(exp.action)
        labels.append(exp.label)
        nextStates.append(exp.nextState)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
    
    return states, actions, labels, nextStates


def getLastExpIndex(folder):
    '''Get the index of the last experience.'''
    files = os.listdir(folder)
    if len(files) == 0:
        return 0

    last = files[-1]
    index = parseExpName(last)[0]
    
    return int(index)


#### Train model function

In [35]:

def evaluateLabels(model, states, actions, labels, sample=0):
    
    if sample > 0:
        sample = min(sample, len(states))
        indexes = random.sample(range(len(states)), sample)
        states = states[indexes]
        actions = actions[indexes]
        labels = labels[indexes]
        
    res = model.predict(states)   
    
    sqErr = [(res[i][actions[i]] - labels[i])**2 for i, out in enumerate(res)]
    meanSqErr = np.mean(sqErr)
    
    return meanSqErr

    
def moveString(m):
    if m == RIGHT:
        return 'right'
    elif m == DOWN:
        return 'down '
    elif m == LEFT:
        return 'left '
    elif m == UP:
        return 'up   '
    else:
        return 'none '
    
    
def getMoveOutputEps(model, screens, eps):
    '''Get the move from the model which produces move scores as outputs, with an epsilon probability for exploration.'''
    
    # If there is a model
    if model != False:        
        p = random.random()
        
        # If exploitation was chosen
        if p > eps:
            res = model.predict(np.asarray([screens]))
            m = np.argmax(res)
            print('Model:', moveString(m), end='\r')

            return m
   
    # If there is no model or exploration was chosen
    m = int(MOVES * random.random())
    print('Rando:', moveString(m), end='\r')
    
    return m 


def copyModel(model, func):
    '''Copy a model and its weights.'''
    newModel = func()
    newModel.set_weights(model.get_weights())
    
    return newModel
    

def trainModelOnline(memory, 
                     model,
                     batchSize=32, 
                     batches=1,
                     file='data/models/model.h5',
                     status=True
                     ):
    '''Train a model online, from the experience memory. 
    Saves the model at the end.
    
    Parameters
    ----------
    memory: the experience memory
    model: the model to be trained
    batchSize: the training batch size
    batches: number of batches to train on
    file: the model file, used for saving
    status: if true, print the status of the training
    '''
        
    # Sample experiences from the memory
    samples = sampleMemory(memory, batches * batchSize, 5, 10, 1)
    batchSize = min(batchSize, len(samples))
    
    # For each batch
    bStart = 0
    
    while bStart < len(samples):
        
        bStop = min(bStart + batchSize, len(samples))
    
        states = []
        targets = []
        
        # For each experience
        while bStart < bStop: 
            e = samples[bStart]
            target = e.label
            
            if target != DEAD_REWARD:
                target = e.label + GAMMA * np.amax(model.predict(np.asarray([e.nextState])))

            fullTarget = model.predict(np.asarray([e.state]))
            fullTarget[0][e.action] = target

            targets.append(fullTarget[0])
            states.append(e.state)
            
            bStart += 1

        # Fit the model with the batch
        targets = np.asarray(targets)

        model.train_on_batch(np.asarray(states), targets)
        
        bStart = bStop
        
    model.save(file)
                
    return model


def trainModelOffline(states, actions, labels, nextStates, 
                      epochs, 
                      batchSize=32, 
                      file='data/models/model.h5', 
                      load=False,
                      status=True,
                      modelFunc=nn_model_actionoutput
                     ):
    '''Train a model offline. 
    Can load or generate a new model. Saves the model after each epoch.
    Implements Fixed Q-target.
    
    Parameters
    ----------
    states: the list of batches of images representing the current states
    actions: the list of actions taken
    labels: the list of rewards obtained
    nextStates: the list of batches of images representing the next states
    epochs: the number of training epochs
    batchSize: the training batch size
    file: the model file, used for saving and loading
    load: if true, the model is loaded from the file, otherwise a new model is generated
    status: if true, print the status of the training
    modelFunc: the type of model to use, given by the model creation function
    '''
    
    if load:
        model = load_model(file)
        print('Model loaded.')
    else:
        model = modelFunc()
        
    # Initialize the target model
    targetModel = copyModel(model, modelFunc)

    dataSize = len(states)
    
    # For each epoch
    for i in range(epochs):
        if status:
            print('Epoch', i+1)
        
        time1 = time.time()
    
        batchStart = 0
        batchEnd = min(batchSize, dataSize)
    
        # For each batch
        while batchStart < batchEnd:

            targets = []
            
            b = batchStart
            # For each experience of the batch, recalculate the targets according to the Q algorithm
            while b < dataSize and b < batchEnd:
                # Get the features of the experience
                s = states[b : b+1]
                a = actions[b]
                l = labels[b]
                n = nextStates[b: b+1]
                
                target = l
                if l != DEAD_REWARD:
                    target = l + GAMMA * np.amax(targetModel.predict(n))
                
                fullTarget = model.predict(s)
                fullTarget[0][a] = target
                
                targets.append(fullTarget[0])
                
                # Clone target model
                if (i * len(states) + b) % CLONE_STEPS == 0:
                    targetModel = copyModel(model, modelFunc)
                
                b += 1
            
            if status and batchStart % (batchSize * 2) == 0:
                print(batchStart, '/', dataSize, end='\r')
                
            # Fit the model with the batch
            targets = np.asarray(targets)
            model.train_on_batch(states[batchStart:batchEnd], targets)

            batchStart = batchEnd
            batchEnd = min(batchEnd + batchSize, dataSize)
          
        if status:  
            print(dataSize, '/', dataSize)
            print("  MSE = %.2f" % evaluateLabels(model, states, actions, labels, sample=50))
        
        time2 = time.time()
        
        if status:
            print("  Duration: %.2f s" % (time2- time1))
            
        # Save model every 10 epochs
        if i % 2 == 0:
            model.save(file)  
      
    model.save(file)  
    
    return model

### Play game

In [23]:
def play(games,
         train=False,
         useModel=False,
         saveExp=False,
         modelFile='data/models/model.h5',
         saveFolder=EXP_FOLDER,
         modelFunc=nn_model_actionoutput):
    
    '''Makes a move every second tick. The previous action does not influence the current action.'''
    
    global browser
    
    e = EPS_INIT
    
    expIndex = getLastExpIndex(saveFolder) + 1
    
    url = 'https://playsnake.org/'
#     browser = webdriver.Chrome(executable_path='D:/Libraries/Drivers/chromedriver_win32/chromedriver.exe')
    browser = webdriver.Chrome(executable_path='C:/Personal/Proiecte/Cod/_libs/Python/selenium_chromedriver73_win32/chromedriver.exe')
    browser.get(url) 
    memory = []
    
    # Load model
    if useModel == True:
        curModel = load_model(modelFile)
    elif train == True:
        curModel = modelFunc()
    else:
        curModel = False

    for g in range(games):

        score = 0
        screenIndex = 0
        time1 = time.time()
    
        episode = []        
        finalScore = 0

        # Select the level
        selectLevel(1)        
        while getState() == 'countdown':
            pass  
        board = getBoard()      

        clickBoard(board)
        screens = deque([], 4)
        
        index = 0
        
        while getState() == 'playing' or getState() == 'paused':
                
            # Get current screenshot
            screens.append(getScreen())            

            # Every two screens
            if index % 2 == 1:

                # Calculate the reward for the previous move
                r = STILL_ALIVE_REWARD
                if getScore() - score > 0:
                    r = SCORE_REWARD 

                score = getScore()
                finalScore = max(finalScore, score)  
                      
                # Form the current state
                state = [screens[-2], screens[-1]]
            
                if index > 2:
                    # Save experience
                    prevState = [screens[0], screens[1]]
                    exp = Experience(prevState, m, r, state)
                    episode.append(exp)  
                
                # Get the current move
                m = getMoveOutputEps(curModel, state, e)
            
                # Send the move to the board
                clickBoard(board)
                makeMove(board, m)
                clickBoard(board) 
            
            else:
                # Move to next screenshot
                clickBoard(board)
                clickBoard(board) 

            index += 1
        
        # Add last experience
        if index % 2 == 1:
            state = [screens[3], screens[3]]
            prevState = [screens[1], screens[2]]
            exp = Experience(prevState, m, DEAD_REWARD, state)
            episode.append(exp) 
            
        # Update last experience
        else:
            exp = episode[-1]
            exp.label = DEAD_REWARD
            
        
        time2 = time.time()        
        print("%d: %.2f s, %d points" % (g, time2-time1, finalScore)) 
        
        # Save experiences
        if saveExp:
            for exp in episode:
                exp.save(saveFolder, expIndex)
                expIndex += 1  
        
        # Only save experiences with positive rewards
        else:
            for exp in episode:
                if exp.label > 0:
                    exp.save(saveFolder, expIndex)
                    expIndex += 1  
            
        if train:     
            # Add the episode to the memory
            rememberExp(memory, episode)

            # Retrain the model            
            curModel = trainModelOnline(memory, model=curModel, file=modelFile, batches=5)
        else:
            time.sleep(1)            
            
        if e > EPS_MIN:
            e = e * EPS_DECAY

    browser.quit()
    
    
    ### DEBUGGING
    return episode



In [55]:
# Train online
# EPS_INIT = 0.1
# EPS_MIN = 0.1
# EPS_DECAY = 0.999
# MAX_MEMORY = 1000
# episode = play(500,
#                train=True,
#                useModel=True,
#                modelFile='data/models/model_cnv3_lin_drp00_lr0001_e150_s3000_o1141.h5',
# #                saveExp=True
#               )


# Save images
# episode = play(50,
#                saveExp=True   
#               )


# Play with model
EPS_INIT = 0.9
episode = play(500,
               useModel=True, 
               modelFile='data/models/model_fqt_deepmind_mse_lr001_g9_e100_s3000.h5',
#                saveExp=True
#                train=True
              )


# Train offline
# states, actions, labels, nextStates = readSamples(sampleFiles(3000))
# # s2, a2, l2, n2 = readSamples(sampleFiles(50))
# model = trainModelOffline(states, actions, labels, nextStates,
#                           modelFunc=nn_model_deepmind,
#                           epochs=100, 
#                           load=False, 
#                           file='data/models/model_fqt_deepmind_mse_lr001_g9_e100_s3000.h5')

0: 13.21 s, 0 points
1: 24.60 s, 0 points
2: 16.41 s, 0 points
3: 16.12 s, 0 points
4: 12.75 s, 0 points
5: 17.03 s, 0 points
6: 27.35 s, 0 points
7: 33.22 s, 0 points
8: 19.88 s, 0 points
9: 11.09 s, 0 points
10: 45.16 s, 0 points
11: 39.40 s, 0 points
12: 19.66 s, 0 points
13: 59.45 s, 0 points
14: 36.46 s, 0 points
15: 113.45 s, 0 points
16: 40.28 s, 0 points
17: 48.86 s, 0 points
18: 6.96 s, 0 points
19: 38.81 s, 5 points
20: 44.32 s, 5 points
21: 20.62 s, 0 points
22: 15.16 s, 0 points
23: 13.25 s, 0 points
24: 24.57 s, 0 points
25: 49.24 s, 5 points
26: 12.16 s, 0 points
27: 14.02 s, 0 points
28: 40.04 s, 0 points
29: 22.71 s, 0 points
30: 9.81 s, 0 points
31: 11.60 s, 0 points
32: 16.85 s, 0 points
33: 73.19 s, 25 points
34: 31.28 s, 0 points
35: 38.07 s, 0 points
36: 29.47 s, 0 points
37: 13.44 s, 0 points
38: 26.22 s, 0 points
39: 24.81 s, 0 points
40: 72.94 s, 0 points
41: 19.79 s, 0 points
42: 12.33 s, 0 points
43: 15.94 s, 0 points
44: 13.94 s, 0 points
45: 31.48 s, 0 point

360: 10.90 s, 0 points
361: 136.27 s, 0 points
362: 23.71 s, 0 points
363: 38.58 s, 0 points
364: 29.56 s, 0 points
365: 110.24 s, 0 points
366: 133.72 s, 0 points
367: 38.63 s, 0 points
368: 37.30 s, 0 points
369: 103.63 s, 5 points
370: 35.80 s, 0 points
371: 35.18 s, 0 points
372: 38.39 s, 0 points
373: 23.31 s, 0 points
374: 90.94 s, 0 points
375: 77.82 s, 0 points
376: 58.82 s, 0 points
377: 17.61 s, 0 points
378: 33.12 s, 0 points
379: 31.33 s, 0 points
380: 99.15 s, 0 points
381: 18.96 s, 0 points
382: 61.49 s, 0 points
383: 74.56 s, 5 points
384: 13.20 s, 0 points
385: 16.91 s, 0 points
386: 14.12 s, 0 points
387: 112.36 s, 0 points
388: 38.38 s, 0 points
389: 69.52 s, 0 points
390: 44.63 s, 0 points
391: 25.15 s, 0 points
392: 57.26 s, 0 points
393: 15.93 s, 0 points
394: 43.41 s, 0 points
395: 62.65 s, 0 points
396: 252.96 s, 5 points
397: 19.83 s, 0 points
398: 31.50 s, 0 points
399: 91.21 s, 0 points
400: 189.12 s, 0 points
401: 111.99 s, 5 points
402: 17.46 s, 0 points
403

#### Evaluate

In [65]:
# Print model structure
from keras.utils.vis_utils import plot_model

model = load_model('data/models/model.h5')
plot_model(model, show_shapes=True, to_file='data/models/model.png')

In [61]:
for i, e in enumerate(episode):
    for j, s in enumerate(e.state):
        saveImage(s, 'data/shots2/e' + str(i) + '_c' + str(j) + '_' + moveString(e.action) + '_' + str(e.label) + '.png')
    for j, s in enumerate(e.nextState):
        saveImage(s, 'data/shots2/e' + str(i) + '_n' + str(j) + '.png')

In [53]:
# # s2, a2, l2, n2 = readSamples(sampleFiles(50))

# model = load_model('data/models/model_cnv3_lin_drp00_lr0001_e150_s3000_o2700.h5')

# for i, state in enumerate(s2):
#     saveImage(state[1], 'data/shots2/s' + str(i) + '.png')
# #     print(model.predict(np.asarray([state])))